In [ ]:
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

top_movies = requests.get('http://www.imdb.com/chart/top')

with open ('top_movies.html', 'wb') as f:
    f.write(top_movies.content)
    
with open('top_movies.html', 'r') as f:
    content = BeautifulSoup(f.read())

content

main_table = content.find('tbody',{'class':'lister-list'}) 

movie_ids = []

for movie in main_table.find_all('td', {'class': 'ratingColumn'}):
    for div_tag in movie.find_all('div'):
        try:
            movie_ids.append(div_tag['data-titleid'])
        except:
            pass 

len(movie_ids)

movie_ids[0:250]

requests.get('http://www.omdbapi.com/?i=tt0111161&r=json')

movie_rows = []

import time

for movie in movie_ids[0:250]:
    r = requests.get('http://www.omdbapi.com/?i=%s&r=json' % movie)
    json_content = json.loads(r.text)
    movie_rows.append(json_content)
    time.sleep(1)

pd.DataFrame(movie_rows)

df = pd.DataFrame(movie_rows)

df

df.to_csv('imdb.csv', sep='\t', encoding='utf-8') 

df.head()

df.columns

df['Awards']

df.isnull().any()

df.isnull().sum().sum()

del df['Poster']

del df['Plot']

df.head()

#clean up Runtime

def erase(text):
    text = text.replace('min','')
    return int(text)

df['Runtime'] = df['Runtime'].apply(erase)

df['Runtime']

plt.hist(df['Runtime'])

del df['Response']

df['imdbVotes']

#clean up Votes..not sure if I want to use these

def omit(commas):
    commas = commas.replace(',','')
    return int(commas)

df['imdbVotes'] = df['imdbVotes'].apply(omit)

df['imdbVotes']

plt.hist(df['imdbVotes'])

df.drop('Type', axis=1, inplace = True)

df.drop('Metascore', axis=1, inplace = True)

df.drop('Released', axis=1, inplace = True)

df.drop('imdbID', axis=1, inplace = True)

df.head()

df['Year'].describe()

def categorize(years):   
    try: 
        if '192' in years:
            years = 1920
        elif '193' in years:
            years = 1930
        elif '194' in years:
            years = 1940
        elif '195' in years:
            years = 1950
        elif '196' in years:
            years = 1960
        elif '197' in years:
            years = 1970
        elif '198' in years:
            years = 1980
        elif '199' in years:
            years = 1990
        elif '200' in years:
            years = 2000
        else:
            years = 2010
        return years 
    except:
        pass

df['Decade'] = df['Year'].apply(categorize)

df['Decade'].head()

df['Decade'].value_counts()

plt.hist(df['Decade'])

df['Rated'].value_counts()

#convert ratings to dummy variables
df_dummies = pd.get_dummies(df['Rated'])

dfwd = pd.concat([df, df_dummies], axis=1)

dfwd.head()

df['Director'].value_counts()

def top(directors):   
    if 'Kubrick' in directors:
        directors = 1
    elif 'Hitchcock' in directors:
        directors = 1
    elif 'Nolan' in directors:
        directors = 1
    elif 'Spielberg' in directors:
        directors = 1
    elif 'Scorsese' in directors:
        directors = 1
    else:
        directors = 0
    return directors

dfwd['Top_Directors'] = dfwd['Director'].apply(top)

dfwd['Top_Directors'].value_counts()

dfwd['Genre'].value_counts()

def popular(genre):   
    if 'Drama' in genre:
        genre = 1
    else:
         genre = 0
    return genre

dfwd['Popular_Genre'] = dfwd['Genre'].apply(popular)

dfwd['Popular_Genre'].value_counts()

dfwd['Country'].head()

def common(country):   
    if country == 'USA':
        country = 1
    else:
        country = 0
    return country

dfwd['Common_Country'] = dfwd['Country'].apply(common)

dfwd['Common_Country'].value_counts()

del dfwd['Actors']

del dfwd['Awards']

del dfwd['Writer']

del dfwd['Title']

dfwd['Language'].value_counts()

def prevalent(language):   
    if language == 'English':
        language = 1
    else:
        language = 0
    return language

dfwd['Prevalent_Language'] = dfwd['Language'].apply(prevalent)

dfwd['Prevalent_Language'].value_counts()

dfwd.dtypes

#time to model!

x = dfwd[['Top_Directors', 'Popular_Genre', 'R', 'PG-13', 'G', 'PG', 'NOT RATED', 'N/A', 'APPROVED', 'Runtime', 'Decade']]
y = dfwd['imdbRating']

x_train, x_test, y_train, y_test = train_test_split(x,y)

rf = RandomForestClassifier()

rf.fit(x_train, y_train)

rf.score(x_train,y_train)

rf.score(x_test, y_test)

rfr = RandomForestRegressor()

rfr.fit(x_train, y_train)

rfr.score(x_train,y_train)

rfr.score(x_test, y_test)

#Do Grid Search with Random Forest Classifier only (since the Regressor score sucks)
#Choose parameters

parameters = [{'n_estimators': [10, 25, 50, 100, 500], 'max_depth': [2, 3, 5, 7, 9],
                     'max_features': [0.25, 0.5, 0.75, 1.0]}]

gridsearch = GridSearchCV(rf, param_grid=parameters, cv=7, n_jobs=-1)

gridsearch.fit(x_train,y_train)

gridsearch.best_estimator_

gridsearch.best_estimator_.fit(x_train, y_train) 

gridsearch.best_estimator_.score(x_train, y_train)

gridsearch.best_estimator_.score(x_test, y_test)






















